In [ ]:
spark

In [ ]:
import pandas as pd
from google.cloud import storage
from io import BytesIO
from datetime import datetime, date

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count, udf, to_date, year, month, date_format, size, split, dayofweek
from pyspark.sql import functions as F

In [ ]:
bucket_name = 'my-bigdataproject-jg'
gs_path  = f'gs://{bucket_name}/'
landing_folder = 'landing/'
cleaned_folder = 'cleaned/'
destination_folder = 'code_and_models/'

storage_client = storage.Client() 
bucket = storage_client.get_bucket(bucket_name)

In [ ]:
spark = SparkSession.builder.master("local[*]") \
    .appName("Taxi Demand Prediction") \
    .getOrCreate()

In [ ]:
weather_files = gs_path + landing_folder + 'weather_data_*_2023.parquet'
weather_df = spark.read.parquet(weather_files)
weather_df.show(15)

In [ ]:
taxi_files = gs_path + landing_folder + 'yellow_tripdata_2023_*.parquet'
taxi_df = spark.read.parquet(taxi_files)